In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
transformations = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
data = datasets.ImageFolder('../input/hotel-id-2021-fgvc8/train_images', transform=transformations)

In [4]:
train_ratio=0.8

train_size = int(len(data)*train_ratio)
val_size = len(data) - train_size

train_set, val_set = torch.utils.data.random_split(data, [train_size, val_size])

In [5]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=32, shuffle=True)

In [6]:
model = models.densenet161(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth


  0%|          | 0.00/110M [00:00<?, ?B/s]

In [7]:
import os, os.path
DATA_PATH = '../input/hotel-id-2021-fgvc8/train_images'
len([name for name in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, name))])

88

In [8]:
classifier_input = model.classifier.in_features
num_labels = len([name for name in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, name))])

# ここは画像認識の構造がよく理解できるようになる
# 詳細1: https://konchangakita.hatenablog.com/entry/2020/05/03/220000
# 詳細2: https://www.javaer101.com/article/1077460.html
# nn.ConV2d(1,6,5): (入力チャンネル、出力チャンネル、カーネルサイズ), 詳細: https://www.atmarkit.co.jp/ait/articles/2005/29/news029_2.html
# torch.nnの機能まとめ: https://pytorch.org/docs/stable/nn.html#linear-layers
# 畳み込み→活性化関数→プーリング→畳み込み→活性化関数→プーリング→全結合
"""
classifier = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=64, 
                                     kernel_size=(5,5), stride=(1,1)),
                           nn.ReLU(),
                           nn.MaxPool2d(kernel_size=2, stride=2),
                           nn.Conv2d(64, 128, 5),
                           nn.ReLU(),
                           nn.MaxPool2d(2, stride=1),
                           nn.Linear(4*4*128, 1024),     # 4×4はcv2のカーネルサイズ5×5をプーリングした結果のものであり、4*4*128はすべての入力特徴量の数値である
                           nn.ReLU(),
                           nn.Linear(1024, num_labels),
                           nn.LogSoftmax(dim=1)
                           )
model.classifier = classifier
"""
classifier = nn.Sequential(nn.Linear(classifier_input, 1024),
                           nn.ReLU(),
                           nn.Linear(1024, 512),
                           nn.ReLU(),
                           nn.Linear(512, num_labels),
                           nn.LogSoftmax(dim=1)
                          )
model.classifier = classifier

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [10]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters())

In [ ]:
epochs = 1
for epoch in range(epochs):
    train_loss = 0
    val_loss = 0
    accuracy = 0
    
    model.train()
    counter = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model.forward(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*inputs.size(0)
        counter += 1
        print(counter, '/', len(train_loader))
    
    model.eval()
    counter = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            output = model.forward(inputs)
            valloss = criterion(output, labels)
            val_loss = valloss.item()*inputs.size(0)
            output = torch.exp(output)
            top_p, top_class = output.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            counter += 1
            print(counter,'/',len(val_loader))
            
    train_loss = train_loss / len(train_loader.dataset)
    valid_loss = val_loss / len(val_loader.dataset)
    print('Accuracy: ', accuracy / len(val_loader))
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))

1 / 2439
2 / 2439
3 / 2439
4 / 2439
5 / 2439
6 / 2439
7 / 2439
8 / 2439
9 / 2439
10 / 2439
11 / 2439
12 / 2439
13 / 2439
14 / 2439
15 / 2439
16 / 2439
17 / 2439
18 / 2439
19 / 2439
20 / 2439
21 / 2439
22 / 2439
23 / 2439
24 / 2439
25 / 2439
26 / 2439
27 / 2439
28 / 2439
29 / 2439
30 / 2439
31 / 2439
32 / 2439
33 / 2439
34 / 2439
35 / 2439
36 / 2439
37 / 2439
38 / 2439
39 / 2439
40 / 2439
41 / 2439
42 / 2439
43 / 2439
44 / 2439
45 / 2439
46 / 2439
47 / 2439
48 / 2439
49 / 2439
50 / 2439
51 / 2439
52 / 2439
53 / 2439
54 / 2439
55 / 2439
56 / 2439
57 / 2439
58 / 2439
59 / 2439
60 / 2439
61 / 2439
62 / 2439
63 / 2439
64 / 2439
65 / 2439
66 / 2439
67 / 2439
68 / 2439
69 / 2439
70 / 2439
71 / 2439
72 / 2439
73 / 2439
74 / 2439
75 / 2439
76 / 2439
77 / 2439
78 / 2439
79 / 2439
80 / 2439
81 / 2439
82 / 2439
83 / 2439
84 / 2439
85 / 2439
86 / 2439
87 / 2439
88 / 2439
89 / 2439
90 / 2439
91 / 2439
92 / 2439
93 / 2439
94 / 2439
95 / 2439
96 / 2439
97 / 2439
98 / 2439
99 / 2439
100 / 2439
101 / 24

In [ ]:
model.eval()

In [ ]:
def process_image(image_path):
    img = Image.open(image_path)
    width, height = img.size
    img.resize((255, int(255*height/width)) if width < height else (int(255*width/height), 255))
    
    left = (224 - width) / 2
    top = (224 - height) / 2
    right = (224 + width) / 2
    bottom = (224 + height) /2
    img = img.crop((left, top, right, bottom))
    
    img = np.array(img)
    
    img = img.transpose((2,0,1))   # HWC → CHW    (推論エンジンのフォーマットはCHW型のため変換が必要)
    img = img/255
    
    img[0] = (img[0] - 0.485) / 0.229
    img[1] = (img[1] - 0.456) / 0.224
    img[2] = (img[2] - 0.406) / 0.225
    
    img = img[np.newaxis,:]
    
    image = torch.from_numpy(img)
    image = image.float()
    return image

In [ ]:
def predict(image, model):
    output = model.forward(image)
    output = torch.exp(output)
    probs, classes = output.topk(1, dim=1)
    return probs.item(), classes.item()

In [ ]:
def show_image(image):
    image = image.numpy()
    
    image[0] = image[0]*0.226 + 0.445
    
    fig = plt.figure(figsize=(25,4))
    plt.imshow(np.transpose(image[0], (1,2,0)))

In [ ]:
"""
TEST_PATH = 
image_file = []
for name_dir in os.listdir(DATA_PATH):
    dir_path = os.path.join(DATA_PATH, name_dir)
    for name_img in dir_path:
        img_path = os.path.join(dir_path, name_img):
        image_file.append(img_path)
"""

In [ ]:
TEST_PATH = '../input/hotel-id-2021-fgvc8/test_images'
test_file =  []
for name in os.listdir(TEST_PATH):
    test_image = os.path.joint(TEST_PATH, name)
    test_file.append(test_image)

In [ ]:
class_file = []
for image_path in test_file:
    image = process_image(image_path)
    top_prob, top_class = predict(image, model)
    show_image(image)
    print("The model is ", top_prob*100, "% certain that the image has a predicted class of ", top_class)
    class_file.append(top_class)

In [ ]:
cla_to_label = {}
for cla, label in zip(range(num_labels), os.listdir(DATA_PATH)):
    cla_to_label[cla] = label

In [ ]:
hotel_data = pd.read_csv('../input/hotel-id-2021-fgvc8/train.csv')
submission_data = pd.read_csv('../input/hotel-id-2021-fgvc8/sample_submission.csv')

In [ ]:
hotel_data.drop(['image', 'timestamp'], axis=1, inplace=True)

In [ ]:
h_id = ''
counter = 0
for key, value in cla_to_label.items():
    hotelID = hotel_data[hotel_data['chain'] == value]['hotel_id']
    for Id in hotelID:
        h_id = h_id + Id + ' '
    submission_data['hotel_id'][counter] = h_id
    counter += 1

In [ ]:
submission_data.to_csv('./kc_Hotel-ID.csv')